# Import Section
---

In [1]:
import regex as re
import shutil
import os
from subprocess import Popen
import subprocess


import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from ipywidgets import AppLayout, Button, Layout, Box, FloatText, Textarea, Dropdown, Label, IntSlider
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

# Widgets Control Section
---

In [12]:
class train_config_and_cmds_widgets():
    def __init__(self):
        
        self.tflite_file_loc = ""
        
        form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
        )
        
        ### data exist ###
        self.A_de = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.B_de = widgets.Text(value='flower_photos', placeholder='Type something', disabled=False)
        
        form_data_prepare_items = [
            Box([Label(value = 'Data Exist'),   self.A_de], layout=form_item_layout),
            Box([Label(value = 'Dataset Name'), self.B_de], layout=form_item_layout),
        ]
        
        self.form_data_prepare_exist = Box(form_data_prepare_items,layout=Layout(
            display='flex',
            flex_flow='column',
            align_items='stretch',
            width='100%',
        ))
        
        ### data download ###
        # Another flowers dataset
        # https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
        self.A_dp = widgets.Textarea(value='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', 
                                     placeholder='Type something', disabled=False)
        # cats_and_dogs.zip
        self.B_dp = widgets.Text(value='flower_photos.tgz', placeholder='Type something', disabled=False)
        
        form_data_prepare_items = [
            Box([Label(value = 'URL Link'), self.A_dp], layout=form_item_layout),
            Box([Label(value = 'Zip Name'), self.B_dp], layout=form_item_layout),
        ]
        
        self.form_data_prepare_cmd = Box(form_data_prepare_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))
        
        ### train ###
        self.A_ta = widgets.Text(value='MyTask', placeholder='Type something', disabled=False)
        self.E_ta = Dropdown(options=['mobilenet_v1', 'mobilenet_v2', 'mobilenet_v3_mini', 'mobilenet_v3', 'fdmobilenet_w1', 'fdmobilenet_wd2', 'fdmobilenet_wd4', 
        'shufflenet_g1_wd4', 'shufflenet_g3_wd4', 'shufflenet_g1_wd2', 'shufflenet_g3_wd2', 'efficientnetB0', 'efficientnetv2B0'])
        self.O_ta = Dropdown(options=['1: Load Pretrain Model(Not support all setting)', '2: Download Pretrain Model Only', '3: Train From Scratch'])
        self.B_ta = widgets.IntSlider(value=32, min=4, max=192, step=4)
        self.C_ta = widgets.IntSlider(value=224, min=32, max=352, step=32)
        self.M_ta = widgets.FloatSlider(value=0.35, min=0.1, max=1.5, step=0.05)
        self.D_ta = widgets.FloatSlider(value=0.2, min=0.1, max=0.5, step=0.1)
        self.F_ta = widgets.FloatSlider(value=0.2, min=0.0, max=0.5, step=0.1)
        self.G_ta = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.H_ta = widgets.Text(value='10,50', placeholder='Type something', disabled=False)
        self.I_ta = widgets.Text(value='0.001,0.0005', placeholder='Type something', disabled=False)
        self.J_ta = Dropdown(value='3: Transfer and Fine-Tuning training', 
                             options=['1: Show the train data and model', '2: Transfer training', 
                                      '3: Transfer and Fine-Tuning training'])
        self.K_ta = widgets.BoundedIntText(value=40, min=0, max=300, step=1, disabled=False)
        self.L_ta = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        
        self.fine_tune_LA = Label(value = 'Freezing Layers of Fine-Tuning')        
        self.K_ta_box = Box([self.fine_tune_LA, self.K_ta], layout=form_item_layout)
        self.alpha_width_LA = Label(value = 'Alpha Width')
        self.M_ta_box = Box([self.alpha_width_LA, self.M_ta], layout=form_item_layout)
        
        form_train_items = [
            Box([Label(value = 'Project Name'),                          self.A_ta], layout=form_item_layout),
            Box([Label(value = 'Model Name'),                            self.E_ta], layout=form_item_layout),
            Box([Label(value = 'Model Setting'),                         self.O_ta], layout=form_item_layout),
            Box([Label(value = 'Batch Size'),                            self.B_ta], layout=form_item_layout),
            Box([Label(value = 'Image Size'),                            self.C_ta], layout=form_item_layout),
            self.M_ta_box,
            Box([Label(value = 'Validation Percent'),                    self.D_ta], layout=form_item_layout),
            Box([Label(value = 'Test Percent from Val Dataset'),         self.F_ta], layout=form_item_layout),
            Box([Label(value = 'Data Augmentation Enable'),              self.G_ta], layout=form_item_layout),
            Box([Label(value = 'Epochs (Transfer, Fine-Tuning)'),        self.H_ta], layout=form_item_layout),
            Box([Label(value = 'Learning Rate (Transfer, Fine-Tuning)'), self.I_ta], layout=form_item_layout),
            Box([Label(value = 'Switch Mode'),                           self.J_ta], layout=form_item_layout),
            self.K_ta_box,
            Box([Label(value = 'Start to Execute'), self.L_ta], layout=form_item_layout),
        ]
        
        self.form_output_train_cmd = Box(form_train_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))
        
        ### test ###
        self.A_tt = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.B_tt = widgets.BoundedIntText(value=1,min=1,max=100,step=1,disabled=False)
        self.C_tt = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
      
        form_test_items = [
            Box([Label(value = 'Choose the tflite file'), self.A_tt], layout=form_item_layout),
            Box([Label(value = 'Batches for test'),       self.B_tt], layout=form_item_layout),
            Box([Label(value = 'Start to Test'),          self.C_tt], layout=form_item_layout),
        ]
        
        self.form_output_test_cmd = Box(form_test_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        ### convert model cpp ###
        self.A_cm = widgets.Text(value='..\workspace\\catsdogs\\tflite_model', placeholder='Type something', disabled=False)
        self.B_cm = widgets.Text(value='mobilenet_v2_int8quant.tflite', placeholder='Type something', disabled=False)
        self.C_cm = widgets.Text(value='..\workspace\\catsdogs\\tflite_model\\vela', placeholder='Type something', disabled=False)
        self.E_cm = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.D_cm = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.F_cm = widgets.Text(value='cats_and_dogs_filtered', placeholder='Type something', disabled=False)
        self.G_cm = widgets.Checkbox(value=True, disabled=False, indent=False)
      
        form_convert_items_paths =  [
            Box([Label(value = 'Choose the tflite file'), self.E_cm], layout=form_item_layout),
            Box([Label(value = 'MODEL SRC DIR'),          self.A_cm], layout=form_item_layout),
            Box([Label(value = 'MODEL SRC FILE'),         self.B_cm], layout=form_item_layout),
            Box([Label(value = 'GEN SRC DIR'),            self.C_cm], layout=form_item_layout)
        ]
        
        form_convert_items_label =  [
           Box([Label(value = 'Dataset Name'), self.F_cm], layout=form_item_layout),
            Box([Label(value = 'Labels Including'), self.G_cm], layout=form_item_layout),
        ]
        
        form_convert_items = [
            Box(form_convert_items_paths, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',    
            border='dotted 3px lightblue',
            align_items='stretch',
            width='70%')),
            Box(form_convert_items_label, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',
            border='dotted 3px lightblue',
            align_items='stretch',
            width='70%')),
            Box([Label(value = 'Convert to cpp & Vela'), self.D_cm], layout=form_item_layout),
        ]
        
        self.form_output_convert_cmd = Box(form_convert_items, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='70%',
        ))
        
    def move_allfiles(self, src_folder, dst_folder):
        copy_num = 0
        
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  #copy whole folder
                shutil.move(fullpath, dst_folder)
                print("Copy finish: {}".format(f))
    
    def show_headline(self, output):
        html0= widgets.HTML(value = f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)
    
    def show_main(self):   
        
        intro_text = 'Please Choose the setting of data prepare & train'
        htmlWidget = widgets.HTML(value = f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(htmlWidget)
        
        #Create an accordion and put the 2 boxes
        accordion = widgets.Accordion(children=[self.form_data_prepare_cmd,
                                                self.form_output_train_cmd
                                                ]).add_class("parentstyle")
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, 'Data Download Setting')
        accordion.set_title(1, 'Configure the Training')
        
        # Create a box combining with 2 elements
        box_data_train = Box([self.form_data_prepare_exist, accordion], layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        #Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[box_data_train, self.form_output_test_cmd, self.form_output_convert_cmd]).add_class("parentstyle")
        tab_contents = ['Train', 'Test', 'Deployment']
        tab.titles = tab_contents
       
        
        #Add a custom style tag to the notebook, you can use dev tool to inspect the class names
        #display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        #accordion.set_title(0, 'Configure the Training')        
        output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))

        # Special observe for MODEL_NAME dependent value updating (fine tune layers)
        def update(*args):
            if self.E_ta.value.count('fdmobile'):
                self.K_ta.value = 6 # bcs the tf2cv structure is combining blocks
                self.K_ta.max = 10
                self.M_ta.layout.visibility = 'hidden'
                self.alpha_width_LA.layout.visibility = 'hidden'
            elif self.E_ta.value.count('shufflenet'):
                self.K_ta.value = 10 # bcs the tf2cv structure is combining blocks
                self.K_ta.max = 17
                self.M_ta.layout.visibility = 'hidden'
                self.alpha_width_LA.layout.visibility = 'hidden'
            elif self.E_ta.value.count('mobilenet_v1'):
                self.K_ta.max = 86
                self.K_ta.value = 40
                self.M_ta.layout.visibility = 'visible'
                self.alpha_width_LA.layout.visibility = 'visible'
            elif self.E_ta.value.count('mobilenet_v2'):
                self.K_ta.max = 154
                self.K_ta.value = 80
                self.M_ta.layout.visibility = 'visible'
                self.alpha_width_LA.layout.visibility = 'visible'
            elif self.E_ta.value.count('mobilenet_v3'):
                self.K_ta.max = 228
                self.K_ta.value = 120
                self.M_ta.layout.visibility = 'visible'
                self.alpha_width_LA.layout.visibility = 'visible'
            elif self.E_ta.value.count('mobilenet_v3_mini'):
                self.K_ta.max = 102
                self.K_ta.value = 50
                self.M_ta.layout.visibility = 'visible'
                self.alpha_width_LA.layout.visibility = 'visible'            
            elif self.E_ta.value.count('efficientnetB0'):
                self.K_ta.max = 238
                self.K_ta.value = 120
                self.M_ta.layout.visibility = 'hidden'
                self.alpha_width_LA.layout.visibility = 'hidden'
            elif self.E_ta.value.count('efficientnetv2B0'):
                self.K_ta.max = 170
                self.K_ta.value = 150
                self.M_ta.layout.visibility = 'hidden'
                self.alpha_width_LA.layout.visibility = 'hidden'        
        self.E_ta.observe(update)

        def act_para(data_exist, URL, zip_n, dataset_n,
                     A_ta, B_ta, C_ta, D_ta, E_ta, F_ta, G_ta, H_ta, I_ta, J_ta, K_ta,
                     A_cm, B_cm, C_cm, F_cm, G_cm, B_tt, 
                     M_ta, O_ta):
        
            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()
                
            if data_exist:
                self.form_data_prepare_cmd.layout.visibility = 'hidden'
            else:
                self.form_data_prepare_cmd.layout.visibility = 'visible'
                
            if J_ta.count('3:'):
                self.K_ta.layout.visibility = 'visible'
                self.fine_tune_LA.layout.visibility = 'visible'
            else:
                self.K_ta.layout.visibility = 'hidden'
                self.fine_tune_LA.layout.visibility = 'hidden'
        
        #------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        #------------------#
        out_inter = widgets.interactive_output(act_para, {'data_exist': self.A_de, 'URL': self.A_dp, 'zip_n': self.B_dp, 
                                                          'dataset_n': self.B_de,
                                                          'A_ta': self.A_ta, 'B_ta': self.B_ta, 'C_ta': self.C_ta,
                                                          'D_ta': self.D_ta, 'E_ta': self.E_ta, 'F_ta': self.F_ta,
                                                          'G_ta': self.G_ta, 'H_ta': self.H_ta, 'I_ta': self.I_ta,
                                                          'J_ta': self.J_ta, 'K_ta': self.K_ta,
                                                          'A_cm': self.A_cm, 'B_cm': self.B_cm, 'C_cm': self.C_cm,
                                                          'F_cm': self.F_cm, 'G_cm': self.G_cm, 'B_tt': self.B_tt,
                                                          'M_ta': self.M_ta, 'O_ta': self.O_ta
                                                          })

        display(tab, out_inter)
        
        #------------------#
        # for labelimg cmd, move to outside of act_para to prevent keep trigering
        #------------------#
        #output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        display(output_widgets)
        
        def on_button_clicked_train(b):
                with output_widgets:
                    clear_output()
                    print("Train. . .")
                    self.run_train()       
        self.L_ta.on_click(on_button_clicked_train)
        
        def on_button_clicked_choose_tflite(b):
                with output_widgets:
                    clear_output()
                    self.choose_tflite()       
        self.E_cm.on_click(on_button_clicked_choose_tflite)
        
        def on_button_clicked_cpp(b):
                with output_widgets:
                    clear_output()
                    print("Convert to cpp & Vela. . .")
                    self.convert_tflu()       
        self.D_cm.on_click(on_button_clicked_cpp)
        
        def on_button_clicked_choose_tflite_test(b):
                with output_widgets:
                    clear_output()
                    self.choose_tflite()       
        self.A_tt.on_click(on_button_clicked_choose_tflite_test)
        
        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.run_test_tflite()           
        self.C_tt.on_click(on_button_clicked_test)

    def choose_tflite(self):
        
        path_ftflite = os.path.join(os.getcwd(), "workspace")
        f_tflite = FileChooser(path_ftflite)
        # Restrict navigation to /Users
        f_tflite.sandbox_path = os.getcwd()
        f_tflite.filter_pattern = ['*.tflite']
        f_tflite.title = f"<b><font color='lightblue'><font size=4>Choose the Tflite for Converting.</b>"
        display(f_tflite)
        
        def act_test():
            work_dir_name = os.getcwd().split("\\")[-1]
            m_src_dir = r".." + f_tflite.selected_path.split(work_dir_name)[-1]
            m_src_tflite  = f_tflite.selected.split("\\")[-1]
            print("The chosen dir: {}".format(m_src_dir))
            print("The chosen tflite: {}".format(m_src_tflite))
            self.A_cm.value = m_src_dir
            self.B_cm.value = m_src_tflite
            self.C_cm.value = os.path.join(m_src_dir, "vela")
            
            
            print("Finish!")
        evt = interact_manual(act_test)
        evt.widget.children[0].description = 'Set this file'  #because there are 3 parameter of the evt
        evt.widget.children[0].button_style = 'primary'
        
    def run_train(self):
        
        sw_num = int(self.J_ta.value.split(":")[0])

        if self.O_ta.value.count('1:'):
            dl_model_set = 0
        elif self.O_ta.value.count('2:'):
            dl_model_set = 1
        else:
            dl_model_set = 2        

        if self.E_ta.value.count('fdmobile') or self.E_ta.value.count('shufflenet'):
            python_file = 'train_tf2cv.py'
            print('Need tf2cv to run this model!')
        else:
            python_file = 'train.py'
        
        %run $python_file --data_exist $self.A_de.value --_URL $self.A_dp.value --zip_name $self.B_dp.value --dataset_name $self.B_de.value \
        --proj_name $self.A_ta.value --BATCH_SIZE $self.B_ta.value --IMG_SIZE $self.C_ta.value --VAL_PCT $self.D_ta.value \
        --MODEL_NAME $self.E_ta.value --TEST_PCT $self.F_ta.value --DATA_AUGM $self.G_ta.value --EPOCHS $self.H_ta.value \
        --LEARNING_RATE $self.I_ta.value --FINE_TUNE_LAYER $self.K_ta.value --switch_mode $sw_num --ALPHA_WIDTH $self.M_ta.value \
        --IMAGENET_MODEL_EN $dl_model_set
        
        path_CMDrecord = os.path.join(os.getcwd(), "workspace", self.A_ta.value, "train_cmd_record.txt")
        with open(path_CMDrecord, "w") as file1:
            open_dir = "cd {} \n".format(os.getcwd())
            cmd = "python {} --data_exist {} --_URL {} --zip_name {} --dataset_name {} --proj_name {} \
--BATCH_SIZE {} --IMG_SIZE {} --VAL_PCT {} --MODEL_NAME {} \
--TEST_PCT {} --DATA_AUGM {} --EPOCHS {} --LEARNING_RATE {} \
--FINE_TUNE_LAYER {} --switch_mode {} --ALPHA_WIDTH {} \
--IMAGENET_MODEL_EN {}".format(python_file, self.A_de.value, self.A_dp.value, self.B_dp.value,
                            self.B_de.value, self.A_ta.value, self.B_ta.value,
                            self.C_ta.value, self.D_ta.value, self.E_ta.value,
                            self.F_ta.value, self.G_ta.value,self.H_ta.value,
                            self.I_ta.value, self.K_ta.value, sw_num, self.M_ta.value, dl_model_set)
            file1.writelines([open_dir, cmd])
        print("Finish !!")
          
    def run_test_tflite(self):
        
        tflite_location = os.path.join((self.A_cm.value).split("..\\")[-1], self.B_cm.value)
        print("The tflite file: {} The number of test batch: {}".format(tflite_location, self.B_tt.value))

        if self.B_cm.value.count('fdmobile') or self.B_cm.value.count('shufflenet'):
            python_file = 'train_tf2cv.py'
            print('Need tf2cv to run this model!')
        else:
            python_file = 'train.py'
        
        %run $python_file --data_exist $self.A_de.value --_URL $self.A_dp.value --zip_name $self.B_dp.value --dataset_name $self.B_de.value \
        --proj_name $self.A_ta.value --BATCH_SIZE $self.B_ta.value --IMG_SIZE $self.C_ta.value --VAL_PCT $self.D_ta.value \
        --MODEL_NAME $self.E_ta.value --TEST_PCT $self.F_ta.value --switch_mode 4 --ALPHA_WIDTH $self.M_ta.value\
        --TFLITE_F $tflite_location --TFLITE_TEST_BATCH_N $self.B_tt.value
        
        print("Finish !!")
        
    def convert_tflu(self):
        
        %run exebat.py --SRC_DIR $self.A_cm.value --SRC_FILE $self.B_cm.value --GEN_DIR $self.C_cm.value
        
        if self.G_cm.value: # create the label C++ source/header files
            # Change to dataset folder
            old_cwd = os.getcwd()
            batch_cwd = os.path.join(old_cwd, "dataset")
            os.chdir(batch_cwd)
            
            %run gen_labels_cpp.py --labels_dataset_name $self.F_cm.value --source_folder_path $self.A_cm.value \
            --header_folder_path $self.A_cm.value
            
            os.chdir(old_cwd)
            print('Finish, the label file is at: {}'.format((old_cwd + self.A_cm.value.split('..')[1])))
            

# Run Section
---
- The detail description of all the parameters and each step meaning is here [meaning](#id-train_evl_monitor)
- In this notebook step, you have alreay finish the dataset prepared. If not, please go to `image_dataset\create_data.ipynb`.

In [14]:
act = train_config_and_cmds_widgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of data prepare & train</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…